# Creating the Channel Funding Transaction

In this section, we’ll create a Lightning channel funding transaction from scratch using Python. We’ll walk through each part of the transaction—how it’s constructed, signed, and how peers exchange messages to share the necessary information to make it happen. We'll test everything using Bitcoin Core in regtest mode.

## Setup

For this exercise we'll need Bitcoin Core. This notebook has been tested with v29.0

Below, set the paths for:

    The bitcoin core functional test framework directory.
    The directory containing taproot-lightning-channels-workshop.

You'll need to edit these next two lines for your local setup.

In [1]:
path_to_bitcoin_functional_test = "/home/pins-dev/Projects/bitcoin/build/test/functional"
path_to_taproot_workshop = "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop"


### Setup bitcoin core test framework

Start up regtest mode, delete any regtest network history so we are starting from scratch.

In [2]:
import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_taproot_workshop)
from functions import *
from functions.test_framework.musig import generate_musig_key
from functions.test_framework.key import generate_bip340_key_pair
from functions.test_framework.script import tagged_hash
from functions.test_framework.address import program_to_witness


import json

# Setup our regtest environment
test = TestShell().setup(
    num_nodes=1, 
    setup_clean_chain=True
)

node = test.nodes[0]

# Create a new wallet and address to send mining rewards so we can fund our transactions
node.createwallet(wallet_name='mywallet')
address = node.getnewaddress()

# Generate 101 blocks so that the first block subsidy reaches maturity
result = node.generatetoaddress(nblocks=101, address=address, called_by_framework=True)

# Check that we were able to mine 101 blocks
assert(node.getblockcount() == 101)

2025-08-30T00:35:39.065000Z TestFramework (INFO): PRNG seed is: 2995932735717337374
2025-08-30T00:35:39.066000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_un6sgnjh


### Setup Alice and Bob Funding Outputs

Create and fund Taproot addresses for Alice and Bob so they have funds to open a simple Taproot channel. The UTXO created for Alice will be the input of the Channel Funding Transaction.

In [3]:
# Create a new Alice key pair for channel funding multisig output
alice_funding_privkey, alice_funding_pubkey = generate_bip340_key_pair()
alice_funding_pubkey_b = alice_funding_pubkey.get_bytes()
alice_funding_privkey_b = alice_funding_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_funding_address = program_to_witness(0x01, alice_funding_pubkey_b)
print(f"Alice funding pubkey: {alice_funding_pubkey_b.hex()}")
print(f"Alice funding privkey: {alice_funding_privkey_b.hex()}")
print(f"Alice funding address: {alice_funding_address}")

# Create a new Bob key pair for channel funding multisig output
bob_funding_privkey, bob_funding_pubkey = generate_bip340_key_pair()
bob_funding_pubkey_b = bob_funding_pubkey.get_bytes()
bob_funding_privkey_b = bob_funding_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
bob_funding_address = program_to_witness(0x01, bob_funding_pubkey_b)
print(f"Bob funding pubkey: {bob_funding_pubkey_b.hex()}")
print(f"Bob funding privkey: {bob_funding_privkey_b.hex()}")
print(f"Bob funding address: {bob_funding_address}")

# Send 1 BTC to Alice and Bob
alice_initial_fund = 1
bob_initial_fund = 1
txid_to_spend = node.sendmany("", {alice_funding_address: alice_initial_fund, bob_funding_address: bob_initial_fund})
print(f"Transaction creating Alice UTXO: {txid_to_spend}")
rawtx_json = node.getrawtransaction(txid_to_spend, True)
print(json.dumps(rawtx_json, indent=2, default=str))

expected_spk = (b"\x51" + b"\x20" + alice_funding_pubkey_b).hex()
index_to_spend = next(
    v["n"] for v in rawtx_json["vout"]
    if v["scriptPubKey"]["hex"] == expected_spk
)
print(f"Alice transaction to spend: {txid_to_spend}:{index_to_spend}")

result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

Alice funding pubkey: 994424e49d0bc2ca18ddcd446aabd038ce5f009f06ec8337e818b1fcd569559d
Alice funding privkey: babbea2ce37893ff8fdeee6017b870a205f16c52a8e0a887947970d72fd153ac
Alice funding address: bcrt1pn9zzfeyap0pv5xxae4zx427s8r897qylqmkgxdlgrzcle4tf2kwslaswka
Bob funding pubkey: 0e35b289dabfed34331adf01fef3c8995d7f39bb6cf5865f9963133969259193
Bob funding privkey: 018189fd03a8af91c6ef9a4f6a06f675022caec84c42e8af14cdd8ea10f6a4ff
Bob funding address: bcrt1ppc6m9zw6hlkngvc6muqlau7gn9wh7wdmdn6cvhuevvfnj6f9jxfs6d4fx4
Transaction creating Alice UTXO: 285c5238d354019e0ed9972914c5a60e35e4a7cc2d0ad159996d8ebee3f40125
{
  "txid": "285c5238d354019e0ed9972914c5a60e35e4a7cc2d0ad159996d8ebee3f40125",
  "hash": "8527b49a511a367c47398a4c6cc1296d8a7ea6f75fa42560f00880a7f168d293",
  "version": 2,
  "size": 289,
  "vsize": 208,
  "weight": 829,
  "locktime": 101,
  "vin": [
    {
      "txid": "46ce84c9a8414ec69e42f27d8868303db566440b240a23509cafdc0dbf17444e",
      "vout": 0,
      "scriptSig": {
    

## Channel Establishment

The "[Extention Bolt](https://github.com/lightning/bolts/pull/995)" for simple taproot channels (work in progress) defines a pathway to create a channel. The messages defined in [BOLT 2](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1) are used to exchange the information needed to create the Channel Funding Transaction, as shown below.


    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+


* (1) open_channel - Alice share its "next_local_nonce"

* (2) accept_channel - Bob share its "next_local_nonce"

* (3) funding_created - Alice samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (4) funding_signed - Bob samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (5) channel_ready - Alice share its "next_local_nonce"

* (6) channel_ready - Bob share its "next_local_nonce"


All the nonces are generated using "NonceGen" algorithm defined in [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki) to ensure it generates nonces in a safe manner.

The pubkeys are sorted using the "KeySort" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To compute the aggregated musig2 public key from the sorted funding_pubkeys use the "KeyAgg" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To construct a musig2 partial signature for the sender's remote commitment use the "Sign" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

### The Channel Funding Transaction

Alice has a 1 BTC UTXO that will serve as the funding transaction input. We’ll now create two outputs:

#### The change output back to Alice

In [4]:

# Create a new Alice key pair output for the change
alice_change_privkey, alice_change_pubkey = generate_bip340_key_pair()
alice_change_pubkey_b = alice_change_pubkey.get_bytes()
alice_change_privkey_b = alice_change_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_change_address = program_to_witness(0x01, alice_change_pubkey_b)
print(f"Alice funding pubkey: {alice_change_pubkey_b.hex()}")
print(f"Alice funding privkey: {alice_change_privkey_b.hex()}")
print(f"Alice funding address: {alice_change_address}")


Alice funding pubkey: b5fa831400d77dae1b122ea279b8e89293ffc2b341f6e1f0c455478415ec2b3d
Alice funding privkey: 560af91ed50f430a80a382f5df7270ad43972b9e6a3a7a19b2151ebdb9691cfe
Alice funding address: bcrt1pkhagx9qq6a76uxcj9638nw8gj2flls4ng8mwruxy24rcg90v9v7sz4szg8


#### The channel musig funding output

The diagram below illustrates how a Taproot channel funding output is constructed.

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +----------+
                    ---| Pagg + T |
                       +----------+
                         ^      ^
                         |      |  +-----------+
                         |       --| T = t * G |<---------
                         |         +-----------+          |
                         |                                |
           +---------------------------------------+    +---+   +-------------------------------+
           |      generate_musig_key(Pa, Pb)       |    | t | = | TaggedHash ("Taptweak", Pagg) |
           +---------------------------------------+    +---+   +-------------------------------+

In [5]:
# Create a new Alice key pair for channel funding multisig output
privkey_alice_musig, pubkey_alice_musig = generate_bip340_key_pair()
print(f"Alice MuSig pubkey: {pubkey_alice_musig.get_bytes().hex()}")
print(f"Alice MuSig privkey: {privkey_alice_musig.get_bytes().hex()}")

# Create a new Bob key pair for channel funding multisig output
privkey_bob_musig, pubkey_bob_musig = generate_bip340_key_pair()
print(f"Bob MuSig pubkey: {pubkey_bob_musig.get_bytes().hex()}")
print(f"Bob MuSig privkey: {privkey_bob_musig.get_bytes().hex()}")

# Generate a 2-of-2 aggregate MuSig key using the pubkeys form Alice and Bob
c_map, agg_pubkey = generate_musig_key([pubkey_alice_musig, pubkey_bob_musig])

# Multiply individual keys with challenges
privalice_c = c_map[pubkey_alice_musig] * privkey_alice_musig
privbob_c = c_map[pubkey_bob_musig] * privkey_bob_musig
pubalice_c = c_map[pubkey_alice_musig] * pubkey_alice_musig
pubbob_c = c_map[pubkey_bob_musig] * pubkey_bob_musig

if agg_pubkey.get_y()%2 != 0:
    privalice_c.negate()
    privbob_c.negate()
    pubalice_c.negate()
    pubbob_c.negate()
    agg_pubkey.negate()

print(f"Aggregate musig pubkey: {agg_pubkey.get_bytes().hex()}")

# Tweak musig public key
# Method: ECPubKey.tweak_add()
tweak = tagged_hash("TapTweak", agg_pubkey.get_bytes())
agg_pubkey_tweaked = agg_pubkey.tweak_add(tweak)
print(f"Tweaked aggregate musig pubkey: {agg_pubkey_tweaked.get_bytes().hex()}")

agg_pubkey_tweaked_b = agg_pubkey_tweaked.get_bytes()

# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
channel_funding_musig_address = program_to_witness(0x01, agg_pubkey_tweaked_b)
print(f"Channel funding musig output address: {channel_funding_musig_address}")

Alice MuSig pubkey: 54bf66778738a751ffd0687f74f4daa14aae44b675aed8042209c5b495ad1da0
Alice MuSig privkey: 3631e6640f9394af621ae2dfc47ce747b155040cc9f0c47beca2df58f772aa7d
Bob MuSig pubkey: 4efaec79b3688870b3d4ff874fdd4b3531b59085ed341980340701ff9565ce77
Bob MuSig privkey: d18ea2a448206ac041c440662989dd59a7da42c9a54fb33981ee9966a32dde22
Aggregate musig pubkey: b38c23d02c05cb75e87fd1846d59dba6163b7277f19570cbd2685d66d96cda37
Tweaked aggregate musig pubkey: 3c459749755da2aa0fddf419c330f7c2d1b6dd9b16de2b94b345ccc4a9c5ea9d
Channel funding musig output address: bcrt1p83zewjt4tk325r7a7svuxv8hctgmdhvmzm0zh99nghxvf2w9a2wsc548az


#### The channel funding transaction

##### The unsigned transaction

In [6]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# Transaction Fee
tx_fee_sat = int(float("0.00000300") * 100000000)

# OUTPUT 1 - Channel Funding Output
# 0.01 BTC to the 2-of-2 multisig Taproot output
output1_value_sat = int(float("0.01") * 100000000)
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output1_spk = bytes.fromhex("51") + varint_len(agg_pubkey_tweaked_b) + agg_pubkey_tweaked_b

# OUTPUT 2 - Change Output
# The rest back to Alice's change address
# 1 BTC - 0.01 BTC - 0.0001 BTC = 0.9899 BTC
output2_value_sat = int(float(alice_initial_fund) * 100000000) - output1_value_sat - tx_fee_sat
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output2_spk = bytes.fromhex("51") + varint_len(alice_change_pubkey_b) + alice_change_pubkey_b

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

unsigned_tx:  02000000012501f4e3be8e6d9959d10a2dcca7e4350ea6c5142997d90e9e0154d338525c280100000000ffffffff0240420f00000000002251203c459749755da2aa0fddf419c330f7c2d1b6dd9b16de2b94b345ccc4a9c5ea9d949de60500000000225120b5fa831400d77dae1b122ea279b8e89293ffc2b341f6e1f0c455478415ec2b3d00000000
{
  "txid": "cb7004ad42d58b208db2efe630d03ad828743c14e36fbea1db39de772b52f74a",
  "hash": "cb7004ad42d58b208db2efe630d03ad828743c14e36fbea1db39de772b52f74a",
  "version": 2,
  "size": 137,
  "vsize": 137,
  "weight": 548,
  "locktime": 0,
  "vin": [
    {
      "txid": "285c5238d354019e0ed9972914c5a60e35e4a7cc2d0ad159996d8ebee3f40125",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "0.01000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "1 3c459749755da2aa0fddf419c330f7c2d1b6dd9b16de2b94b345ccc4a9c5ea9d",
        "desc": "rawtr(3c459749755da2aa0fddf419c330f7c2d1b6dd9b16de2b94b345ccc4

##### The sighash for key path spend

This is the “Schnorr key spend” case: you’re proving knowledge of the (tweaked) internal private key, with no script branch revealed. The message preimage is called msg_digest in [BIP-341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki).

In [7]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index)

input_amount_sat = int(alice_initial_fund * 100_000_000)
input_amounts = input_amount_sat.to_bytes(8, byteorder="little", signed=False)
sha_amounts = sha256(input_amounts)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
spk = bytes.fromhex("51") + varint_len(alice_funding_pubkey_b) + alice_funding_pubkey_b

sha_scriptpubkeys = sha256(
    varint_len(spk)
    + spk
)

sha_sequences = sha256(sequence)

sha_outputs = sha256(outputs) ######

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode())
sighash = sha256(tag_hash + tag_hash + sig_msg)


##### Signing the sighash

In [8]:
aux_rand = b"\x00"*32
signature = alice_funding_privkey.sign_schnorr(sighash, aux_rand) 

##### The signed transaction

Finally the last step is to add the signature to the witness field and she has the signed transaction!

In [9]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(signature)
    + signature
)
print(witness.hex())

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_tx.hex())
# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
print(result)

014046d6da9b8af87411618a5b56892b74417a101ae1baa1c225b66b2478db65f283b8451e7da310ebbadabb903ab0e4d4fcbeb8a1ba84ea7f0b50c5a0a131eed6a6
signed tx:  020000000001012501f4e3be8e6d9959d10a2dcca7e4350ea6c5142997d90e9e0154d338525c280100000000ffffffff0240420f00000000002251203c459749755da2aa0fddf419c330f7c2d1b6dd9b16de2b94b345ccc4a9c5ea9d949de60500000000225120b5fa831400d77dae1b122ea279b8e89293ffc2b341f6e1f0c455478415ec2b3d014046d6da9b8af87411618a5b56892b74417a101ae1baa1c225b66b2478db65f283b8451e7da310ebbadabb903ab0e4d4fcbeb8a1ba84ea7f0b50c5a0a131eed6a600000000
{
  "txid": "cb7004ad42d58b208db2efe630d03ad828743c14e36fbea1db39de772b52f74a",
  "hash": "4053fde296a8e3036eea96cb545d27483c1d99c65bba062c7263eb3adef0a563",
  "version": 2,
  "size": 205,
  "vsize": 154,
  "weight": 616,
  "locktime": 0,
  "vin": [
    {
      "txid": "285c5238d354019e0ed9972914c5a60e35e4a7cc2d0ad159996d8ebee3f40125",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [